## Runnable 的调用机制

### Runnable 基类实现了 __call__ 方法

几乎所有 illufly 的类将 Runnable 作为基类。

Runnable 自己实现了 __call__ 方法，并在这个方法中调用 call 方法。<br>
因为 call 是一个抽象方法，因此要求子类必须实现自己它。

有了 __call__ 方法，你就可以将类的实例当作方法一样使用。

**这样做的好处：**
这很方便，也足够简单，因为你只需要记住 illufly 中的智能体对象只有一个方法，并且你不需要记住名字。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        print("hi")

r = MyRun()
r()

hi



这样，Runnable 就可以通过 __call__ 方法调用自己的 call 方法。

### 使用 EventBlock 交换事件信息流

In [10]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi")

r = MyRun()
r()

hi

## 使用 Runnable 的流式输出

### 默认的 log 处理

默认情况下，Runnable 的 handlers 列表中已经有 log 函数。
因此你可以隐藏调用 log 来打印流式内容。

In [5]:
# 使用 handler 函数的另一种方式：
from illufly.chat import FakeLLM

llm = FakeLLM()
llm("你能帮我写一首关于兔子做梦的四句儿歌?")

这是一个模拟调用!

'这是一个模拟调用!'

In [6]:
# 这与下面的代码等价
from illufly.io import log

llm = FakeLLM(handlers=[log])
llm("你能帮我写一首关于兔子做梦的四句儿歌?")

这是一个模拟调用!

'这是一个模拟调用!'

### 异步处理的 handler

只要 handler 中有一个是异步处理器，你就必须使用异步处理。<br>
alog 是 log 的异步版本。

In [1]:
from illufly.chat import FakeLLM
from illufly.io import alog

llm = FakeLLM(handlers=[alog])
await llm("你能帮我写一首关于兔子做梦的四句儿歌?")

这是一个模拟调用!

'这是一个模拟调用!'

### `illufly.io` 中的 log 和 usage

usage 处理函数用于捕捉生成器结果中的 

In [9]:
from illufly.chat import ChatOpenAI
from illufly.io import log, usage

openai = ChatOpenAI(handlers=[log, usage], verbose=True)
openai("你能帮我写一首关于兔子做梦的四句儿歌?")

清晨静悄悄，兔儿梦中跳。
白雪映夜晚，兔儿梦中藏。
梦里花园美，兔儿跳跃着。
梦醒时惊醒，兔儿又在梦。
{"block_type": "usage", "content": "{\"prompt_tokens\": 35, \"completion_tokens\": 93, \"total_tokens\": 128}", "created_at": "2024-10-05T16:07:38.723463", "calling_info": {"request_id": null, "input": {"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}], "tools": null, "stream": true, "stream_options": {"include_usage": true}}, "output": [{"chunk": "清"}, {"chunk": "晨"}, {"chunk": "静"}, {"chunk": "悄"}, {"chunk": "悄"}, {"chunk": "，"}, {"chunk": "兔"}, {"chunk": "儿"}, {"chunk": "梦"}, {"chunk": "中"}, {"chunk": "跳"}, {"chunk": "。\n"}, {"chunk": "白"}, {"chunk": "雪"}, {"chunk": "映"}, {"chunk": "夜"}, {"chunk": "晚"}, {"chunk": "，"}, {"chunk": "兔"}, {"chunk": "儿"}, {"chunk": "梦"}, {"chunk": "中"}, {"chunk": "藏"}, {"chunk": "。\n"}, {"chunk": "梦"}, {"chunk": "里"}, {"chunk": "花"}, {"chunk": "园"}, {"chunk": "美"}, {"chunk": "，"}, {"chunk": "兔"}, {"chunk": "儿"}, {"chunk": "跳"}, {"chunk": "跃"}, {"chunk": "着"}, {"c

'清晨静悄悄，兔儿梦中跳。\n白雪映夜晚，兔儿梦中藏。\n梦里花园美，兔儿跳跃着。\n梦醒时惊醒，兔儿又在梦。'

## 使用绑定机制传播 handlers

In [1]:
from illufly.types import Runnable
from illufly.chat import FakeLLM
from illufly.io import log

### 定义一个新的 handler

定义 `handler` 必须使用形如 `block,, verbose, **kwargs` 这样的入参结构：

In [2]:
def myhandler(block, verbose=False, **kwargs):
    if block.block_type == 'info':
        print("I got a [info] event, but I wont tell you...")

In [3]:
llm = FakeLLM()
llm("hi")

这是一个模拟调用!

'这是一个模拟调用!'

### 将 provider 中的 handler 传递给 consumer

In [4]:
class myrun(Runnable):
    def call(self, prompt, **kwargs):
        llm = FakeLLM()
        self.bind_consumer(llm)
        resp = llm(prompt)
r = myrun(handlers = [log, myhandler])
r("hi")

I got a [info] event, but I wont tell you...
这是一个模拟调用!

In [5]:
r.consumer_tree

{'provider': <Runnable myrun.4767904816>,
 'consumer_tree': [(<Runnable FakeLLM.4767904144>,
   {},
   {'provider': <Runnable FakeLLM.4767904144>, 'consumer_tree': []})]}

In [8]:
llm.provider_tree

{'consumer': <Runnable FakeLLM.4767898768>, 'provider_tree': []}